In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95ca-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95b1-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95d7-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95c5-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95c6-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95b9-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95c9-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95d0-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95d8-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95c2-8d87-11e8-9daf-6045cb817f5b..JPG
/kaggle/input/fundusimage1000/1000images/0.3.DR1/1ffa95d4-8d87-11e8-9daf-6045cb817f5b..JPG

In [4]:
!pip install torchcam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing install

In [5]:
import os
import glob
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.preprocessing import MultiLabelBinarizer
import timm
from torchcam.methods import GradCAM
import matplotlib.pyplot as plt

In [6]:
# ---------------- CONFIG ----------------
IMG_SIZE = 299
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT_DIR = '/kaggle/input/fundusimage1000/1000images'
MODEL_SAVE_PATH = "/kaggle/working/best_fundus_efficientnetb3.pth"
CSV_OUTPUT_PATH = "/kaggle/working/fundus_predictions.csv"

In [8]:
# ---------------- LABEL SETUP ----------------
class_names = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
mlb = MultiLabelBinarizer()
mlb.fit([class_names])  # Fit on list of all classes

# ---------------- TRANSFORMS ----------------
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

# ---------------- DATASET ----------------
class FundusDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        
        for cls in class_names:
            folder = os.path.join(root_dir, cls)
            imgs = glob.glob(os.path.join(folder, "*.JPG")) + glob.glob(os.path.join(folder, "*.jpg"))
            self.image_paths.extend(imgs)
            self.labels.extend([[cls]] * len(imgs))
        
        self.encoded_labels = mlb.transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        label = torch.FloatTensor(self.encoded_labels[idx])
        if self.transform:
            img = self.transform(img)
        return img, label

In [9]:
# ---------------- DATALOADERS ----------------
dataset = FundusDataset(ROOT_DIR, transform=None)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_set.dataset.transform = train_tfms
val_set.dataset.transform = val_tfms

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE)

# ---------------- MODEL ----------------
model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=len(class_names))
model = model.to(DEVICE)

# ---------------- CAM ----------------
cam_extractor = GradCAM(model, target_layer="conv_head")


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

In [10]:
# ---------------- TRAINING ----------------
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

def train_model():
    best_loss = float('inf')
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        
        # Training phase
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0
        
        # Disable CAM during validation
        cam_extractor._hooks_enabled = False
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        # Re-enable CAM for next training epoch
        cam_extractor._hooks_enabled = True
        
        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(" Saved best model!")

In [11]:
# ---------------- PREDICTION + CSV ----------------
def export_predictions(model, dataloader, csv_path):
    model.eval()
    all_preds, all_labels, all_paths = [], [], []

    with torch.no_grad():
        for batch_imgs, batch_labels in tqdm(dataloader, desc="Exporting Predictions"):
            batch_imgs = batch_imgs.to(DEVICE)
            outputs = model(batch_imgs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            all_preds.extend(probs)
            all_labels.extend(batch_labels.numpy())

    # Get original image paths from validation set
    val_indices = val_set.indices if hasattr(val_set, 'indices') else range(len(val_set))
    all_paths = [dataset.image_paths[i] for i in val_indices]

    df_preds = pd.DataFrame(all_preds, columns=[f"Pred_{c}" for c in class_names])
    df_true = pd.DataFrame(all_labels, columns=[f"True_{c}" for c in class_names])
    df_paths = pd.DataFrame({"Image_Path": all_paths[:len(all_preds)]})

    df_combined = pd.concat([df_paths, df_true, df_preds], axis=1)
    df_combined.to_csv(csv_path, index=False)
    print(f"✅ Predictions saved to {csv_path}")


In [12]:
# ---------------- GRAD-CAM DISPLAY ----------------
def visualize_cam(image_tensor, class_idx):
    model.eval()
    image_tensor = image_tensor.unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        scores = model(image_tensor)

    cam = cam_extractor(class_idx, scores)[0].squeeze().cpu().numpy()

    img_np = image_tensor.cpu().squeeze().permute(1, 2, 0).numpy()
    img_np = (img_np * 0.5) + 0.5  # reverse normalization

    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].imshow(img_np)
    ax[0].set_title("Original")
    ax[0].axis('off')
    
    ax[1].imshow(img_np)
    ax[1].imshow(cam, cmap='jet', alpha=0.5)
    ax[1].set_title(f"Grad-CAM: {class_names[class_idx]}")
    ax[1].axis('off')
    plt.tight_layout()
    plt.show()


In [13]:
# ---------------- RUN ----------------
print(f"Using device: {DEVICE}")
print(f"Found {len(dataset)} images across {len(class_names)} classes")

Using device: cpu
Found 997 images across 40 classes


In [17]:
train_model()

Epoch 1/10: 100%|██████████| 50/50 [10:00<00:00, 12.01s/it]


Epoch 1, Train Loss: 0.0826
Validation Loss: 0.0717
 Saved best model!


Epoch 2/10: 100%|██████████| 50/50 [10:09<00:00, 12.19s/it]


Epoch 2, Train Loss: 0.0467
Validation Loss: 0.0554
 Saved best model!


Epoch 3/10: 100%|██████████| 50/50 [09:43<00:00, 11.67s/it]


...

Epoch 8, Train Loss: 0.0033
Validation Loss: 0.0414


Epoch 9/10: 100%|██████████| 50/50 [09:46<00:00, 11.72s/it]


Epoch 9, Train Loss: 0.0025
Validation Loss: 0.0403


Epoch 10/10: 100%|██████████| 50/50 [09:45<00:00, 11.72s/it]


Epoch 10, Train Loss: 0.0020
Validation Loss: 0.0401
